In [1]:
## Seismic Analysis Impact on NR&C - Test Population
## By: Fervis Lauan
## Date: 09-29-17
##----------------------------------------------------


# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import requests
from census import Census
from us import states


In [2]:
##################################
# Get Test Population
##################################



ziplst=[96130,89406,95501,93117,82520,92247,73013,95422,73020,95540,75964,93555,93561,92284,98807,75652,89433,93212,90275,93436,96150]

ctr=1
for zip in ziplst:   
    yr=2013            
    while yr<=2014:
        try:
            c = Census("85ac64b6b5a9c0901b00329d1ef41f0c53ccfc98", year=yr)
            census_data = c.acs5.get(("NAME",
                                              "B01003_001E", 
                                              "B01002_001E",
                                              "B19013_001E",
                                              "B19301_001E",
                                              "B17001_002E",
                                              "B23025_002E",
                                              "B23025_005E",
                                              "B25077_001E",
                                              "B25064_001E",
                                              "C24010_009E",
                                              "C24010_016E",
                                              "C24010_019E",
                                              "C24010_022E",
                                              "C24010_023E",
                                              "C24010_025E",
                                              "C24010_030E",
                                              "C24010_032E",
                                              "C24010_045E",
                                              "C24010_052E",
                                              "C24010_055E",
                                              "C24010_058E",
                                              "C24010_059E",
                                              "C24010_061E",
                                              "C24010_066E",
                                              "C24010_068E",                         
                                     ), {'for': 'zip code tabulation area:'+str(zip)})
            # Convert to DataFrame
            census_pd = pd.DataFrame(census_data)
            census_pd["Year"]=yr
            # Column Reordering
            census_pd = census_pd.rename(columns={"B01003_001E": "Population", 
                                                  "B01002_001E": "Median Age",
                                                  "B19013_001E": "Household Income",
                                                  "B19301_001E": "Per Capita Income",
                                                  "B17001_002E": "Poverty Count",
                                                  "B23025_002E": "Employment Count",
                                                  "B23025_005E": "Unemployment Count",
                                                  "B25077_001E": "Median Home Value",
                                                  "B25064_001E": "Median Gross Rent",                                      
                                                  "C24010_009E": "Emp Male Arch Engnr",
                                                  "C24010_016E": "Emp Male Health Prac",
                                                  "C24010_019E": "Emp Male Service Occ",
                                                  "C24010_022E": "Emp Male Fire Prev",
                                                  "C24010_023E": "Emp Male Law Enfrcmnt",
                                                  "C24010_025E": "Emp Male BldgGrnd Cleaning",
                                                  "C24010_030E": "Emp Male Ntrl Rsrces Const",
                                                  "C24010_032E": "Emp Male Const Extrctn",                                      
                                                  "C24010_045E": "Emp FMale Arch Engnr",
                                                  "C24010_052E": "Emp FMale Health Prac",
                                                  "C24010_055E": "Emp FMale Service Occ",
                                                  "C24010_058E": "Emp FMale Fire Prev",
                                                  "C24010_059E": "Emp FMale Law Enfrcmnt",
                                                  "C24010_061E": "Emp FMale BldgGrnd Cleaning",
                                                  "C24010_066E": "Emp FMale Ntrl Rsrces Const",
                                                  "C24010_068E": "Emp FMale Const Extrctn",                                                                          
                                                  "NAME": "Name",                                       
                                                  "zip code tabulation area": "Zip",
                                                  "Year":"Year"})

            census_pd["Emp Arch Engnr"]=census_pd["Emp Male Arch Engnr"]+census_pd["Emp FMale Arch Engnr"]
            census_pd["Emp Health Prac"]=census_pd["Emp Male Health Prac"]+census_pd["Emp FMale Health Prac"]
            census_pd["Emp Service Occ"]=census_pd["Emp Male Service Occ"]+census_pd["Emp FMale Service Occ"]
            census_pd["Emp Fire Prev"]=census_pd["Emp Male Fire Prev"]+census_pd["Emp FMale Fire Prev"]
            census_pd["Emp Law Enfrcmnt"]=census_pd["Emp Male Law Enfrcmnt"]+census_pd["Emp FMale Law Enfrcmnt"]
            census_pd["Emp BldgGrnd Cleaning"]=census_pd["Emp Male BldgGrnd Cleaning"]+census_pd["Emp FMale BldgGrnd Cleaning"]
            census_pd["Emp Ntrl Rsrces Const"]=census_pd["Emp Male Ntrl Rsrces Const"]+census_pd["Emp FMale Ntrl Rsrces Const"]
            census_pd["Emp Const Extrctn"]=census_pd["Emp Male Const Extrctn"]+census_pd["Emp FMale Const Extrctn"]

            # Add in Poverty Rate (Poverty Count / Population)
            census_pd["Poverty Rate"] = 100 * census_pd["Poverty Count"].astype(int) / census_pd["Population"].astype(int)

            # Add in Employment Rate (Employment Count / Population)
            census_pd["Unemployment Rate"] = 100 * census_pd["Unemployment Count"].astype(int) / census_pd["Population"].astype(int)
            
            # Final DataFrame
            census_pd = census_pd[["Year",                       
                                   "Name", 
                                   "Zip",
                                   "Population", 
                                   "Median Age", 
                                   "Household Income",
                                   "Per Capita Income", 
                                   "Poverty Count", 
                                   "Poverty Rate",
                                   "Employment Count", 
                                   "Unemployment Rate",
                                   "Median Home Value",
                                   "Median Gross Rent",
                                   "Emp Arch Engnr",
                                   "Emp Health Prac",
                                   "Emp Service Occ",
                                   "Emp Fire Prev",
                                   "Emp Law Enfrcmnt",
                                   "Emp BldgGrnd Cleaning",
                                   "Emp Ntrl Rsrces Const",
                                   "Emp Const Extrctn"
                                   ]]                 

            if ctr==1:
                print("Creating from year "+str(yr)+"...")
                census_pd_test=census_pd
            else:
                print("appending year "+str(yr)+"...")
                census_pd_test=census_pd_test.append(census_pd)
            ctr+=1
        except:
            print("No Data")
        yr+=1
    

Creating from year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
No Data
No Data
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
No Data
No Data
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...
appending year 2013...
appending year 2014...


In [3]:
census_pd_test2013=census_pd_test.loc[census_pd_test["Year"]==2013]
census_pd_test2014=census_pd_test.loc[census_pd_test["Year"]==2014]

In [4]:
census_pd_testmrg=pd.merge(census_pd_test2013, census_pd_test2014, on="Zip")
census_pd_testmrg

,Year_x,Name_x,Zip,Population_x,Median Age_x,Household Income_x,Per Capita Income_x,Poverty Count_x,Poverty Rate_x,Employment Count_x,...,Median Home Value_y,Median Gross Rent_y,Emp Arch Engnr_y,Emp Health Prac_y,Emp Service Occ_y,Emp Fire Prev_y,Emp Law Enfrcmnt_y,Emp BldgGrnd Cleaning_y,Emp Ntrl Rsrces Const_y,Emp Const Extrctn_y
0,2013,ZCTA5 96130,96130,21963.0,35.0,54372.0,18621.0,2847.0,12.962710,6710.0,...,176100.0,919.0,16.0,427.0,1918.0,64.0,1065.0,146.0,294.0,145.0
1,2013,ZCTA5 89406,89406,24572.0,39.2,49830.0,24716.0,3619.0,14.728146,11912.0,...,144900.0,838.0,74.0,522.0,2083.0,283.0,61.0,497.0,1427.0,379.0
2,2013,ZCTA5 95501,95501,23704.0,37.0,37600.0,21122.0,4968.0,20.958488,12342.0,...,247300.0,839.0,111.0,367.0,3070.0,251.0,144.0,436.0,943.0,685.0
3,2013,ZCTA5 93117,93117,54251.0,21.9,56669.0,22722.0,12866.0,23.715692,29618.0,...,631000.0,1450.0,644.0,1015.0,6372.0,541.0,84.0,1306.0,1443.0,655.0
4,2013,ZCTA5 82520,82520,13778.0,39.3,56554.0,24971.0,1549.0,11.242561,7138.0,...,207800.0,696.0,41.0,456.0,1402.0,80.0,67.0,395.0,837.0,484.0
5,2013,ZCTA5 73013,73013,46457.0,34.7,75741.0,38470.0,3761.0,8.095658,24288.0,...,190500.0,1090.0,759.0,2145.0,2498.0,161.0,154.0,329.0,1326.0,673.0
6,2013,ZCTA5 95422,95422,15427.0,41.6,25460.0,16552.0,5484.0,35.548065,6350.0,...,90400.0,825.0,36.0,145.0,1466.0,109.0,59.0,190.0,931.0,317.0
7,2013,ZCTA5 73020,73020,21499.0,40.1,67088.0,29985.0,2051.0,9.539979,10611.0,...,158800.0,902.0,254.0,602.0,1235.0,131.0,118.0,230.0,1349.0,743.0
8,2013,ZCTA5 95540,95540,12877.0,38.7,40751.0,21707.0,2543.0,19.748389,5336.0,...,260700.0,816.0,32.0,204.0,1077.0,58.0,160.0,276.0,604.0,316.0
9,2013,ZCTA5 75964,75964,19648.0,32.8,35827.0,17313.0,6085.0,30.970073,8920.0,...,77100.0,724.0,42.0,404.0,1280.0,39.0,81.0,408.0,1114.0,867.0


In [5]:
#census_pd_testmrg=census_pd_testmrg.loc[(census_pd_testmrg["Population_x"]>=1100) & (census_pd_testmrg["Population_x"]<=35000)]
#census_pd_testmrg

In [6]:
census_pd_testsample=census_pd_testmrg


In [7]:
census_pd_testsample

,Year_x,Name_x,Zip,Population_x,Median Age_x,Household Income_x,Per Capita Income_x,Poverty Count_x,Poverty Rate_x,Employment Count_x,...,Median Home Value_y,Median Gross Rent_y,Emp Arch Engnr_y,Emp Health Prac_y,Emp Service Occ_y,Emp Fire Prev_y,Emp Law Enfrcmnt_y,Emp BldgGrnd Cleaning_y,Emp Ntrl Rsrces Const_y,Emp Const Extrctn_y
0,2013,ZCTA5 96130,96130,21963.0,35.0,54372.0,18621.0,2847.0,12.962710,6710.0,...,176100.0,919.0,16.0,427.0,1918.0,64.0,1065.0,146.0,294.0,145.0
1,2013,ZCTA5 89406,89406,24572.0,39.2,49830.0,24716.0,3619.0,14.728146,11912.0,...,144900.0,838.0,74.0,522.0,2083.0,283.0,61.0,497.0,1427.0,379.0
2,2013,ZCTA5 95501,95501,23704.0,37.0,37600.0,21122.0,4968.0,20.958488,12342.0,...,247300.0,839.0,111.0,367.0,3070.0,251.0,144.0,436.0,943.0,685.0
3,2013,ZCTA5 93117,93117,54251.0,21.9,56669.0,22722.0,12866.0,23.715692,29618.0,...,631000.0,1450.0,644.0,1015.0,6372.0,541.0,84.0,1306.0,1443.0,655.0
4,2013,ZCTA5 82520,82520,13778.0,39.3,56554.0,24971.0,1549.0,11.242561,7138.0,...,207800.0,696.0,41.0,456.0,1402.0,80.0,67.0,395.0,837.0,484.0
5,2013,ZCTA5 73013,73013,46457.0,34.7,75741.0,38470.0,3761.0,8.095658,24288.0,...,190500.0,1090.0,759.0,2145.0,2498.0,161.0,154.0,329.0,1326.0,673.0
6,2013,ZCTA5 95422,95422,15427.0,41.6,25460.0,16552.0,5484.0,35.548065,6350.0,...,90400.0,825.0,36.0,145.0,1466.0,109.0,59.0,190.0,931.0,317.0
7,2013,ZCTA5 73020,73020,21499.0,40.1,67088.0,29985.0,2051.0,9.539979,10611.0,...,158800.0,902.0,254.0,602.0,1235.0,131.0,118.0,230.0,1349.0,743.0
8,2013,ZCTA5 95540,95540,12877.0,38.7,40751.0,21707.0,2543.0,19.748389,5336.0,...,260700.0,816.0,32.0,204.0,1077.0,58.0,160.0,276.0,604.0,316.0
9,2013,ZCTA5 75964,75964,19648.0,32.8,35827.0,17313.0,6085.0,30.970073,8920.0,...,77100.0,724.0,42.0,404.0,1280.0,39.0,81.0,408.0,1114.0,867.0


In [8]:
for index,row in census_pd_testsample.iterrows():
    pcchg=((row["Emp Const Extrctn_y"]+row["Emp Ntrl Rsrces Const_y"])-(row["Emp Const Extrctn_x"]+row["Emp Ntrl Rsrces Const_x"]))/(row["Emp Const Extrctn_x"]+row["Emp Ntrl Rsrces Const_x"])
    census_pd_testsample.loc[index,"Emp Const Extr Change"]=pcchg
    


In [9]:
census_pd_testsample.to_csv("./Resources/seismic_test_emp_pop.csv")

    
    

In [10]:
census_pd_test_ttest=census_pd_testsample[["Zip","Emp Const Extr Change"]]

In [11]:
census_pd_test_ttest

,Zip,Emp Const Extr Change
0,96130,0.094763
1,89406,0.105940
2,95501,-0.138624
3,93117,0.186652
4,82520,0.057646
5,73013,-0.029612
6,95422,0.039134
7,73020,0.078351
8,95540,0.021088
9,75964,-0.126543


In [12]:
census_pd_test_ttest.to_csv("./Resources/seismic_test_emp_pop_ttest.csv")